In [1]:
import numpy as np
import os
import warnings
from functools import partial
import multiprocessing.pool
from six.moves import xrange 
import tensorflow as tf
import math
import sys
sys.path.insert(0, '../')
from utils import *
from create_synthetic_dataset import *

os.environ["CUDA_VISIBLE_DEVICES"]="0"
%load_ext autoreload
%autoreload 2

In [ ]:
def placeholder_inputs():
    """Generate placeholder variables to represent the input tensors.
    Returns:
    images_placeholder: Images placeholder.
    labels_placeholder: Labels placeholder.
    """
    images_placeholder = tf.placeholder(tf.float32, shape=(None,
                                                         model.IM_SIZE,
                                                         model.IM_SIZE,
                                                         model.IM_SIZE,
                                                         model.CHANNELS))
    labels_placeholder = tf.placeholder(tf.int64, shape=(None))
    return images_placeholder, labels_placeholder



In [ ]:
X,y = create_synthetic_dataset()

> /home/vincent/repos/sh01/utils.py(193)copy_template()
-> cube_size = cube.shape[0]
(Pdb) cube
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0

In [ ]:
## Import the model definition:
import s_lri_model as model
#import s_lri_deep2_model as model
#import sse_lri_model as model
#import bispectrum_lri_model as model
#import g_lri_model as model
#import g_ri_model as model
#import conv_separable_model as model
#import g_lri_sep_model as model

## All parameters for training
is_shconv = True # To compare with a normal CNN (shconv=False) 
is_trainable = True # To test freezing the conv weights
is_hn=True # If false: polar-separable
is_augment=False # Right angle rotation augmentation at training
M = 4 # number of orientations (only needed for S-LRI). 1, 4, 24 or 72
nf1 = 2 # number of filters in first layer
batch_size = 16
stride = 1 
lr = 1e-3
max_steps = 50000
degreeMax = 1 # maxmum degree of the SHs
ksize = 7

In [ ]:
list_test_acc = []
for _ in xrange(10):
    n_class = 2
    tf.reset_default_graph()
    # Placeholders    
    xph, yph = placeholder_inputs()
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')

    # Construct model and optimizer
    pred = model.build_model(X = xph,
                            batch_size = batch_size,
                            n_class = n_class,
                            nf1 = nf1,
                            ksize = ksize,
                            stride = stride,
                            is_trainable = is_trainable,
                            degreeMax = degreeMax,
                            is_shconv = is_shconv,
                            is_hn = is_hn,
                            M = M
                            )
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=yph))

    # Evaluation criteria
    correct_pred = tf.equal(tf.argmax(pred, 1), yph)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    # Optimizer
    optim = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.99, beta2=0.9999)
    grads_and_vars = optim.compute_gradients(loss)
    train_op = optim.apply_gradients(grads_and_vars)


    # Configure tensorflow session
    init_global = tf.global_variables_initializer()
    init_local = tf.local_variables_initializer()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.log_device_placement = False

    sess = tf.Session(config=config)
    sess.run([init_global, init_local])

    # shuffle           
    np.random.seed(0)
    idx = np.arange(0,len(X))
    np.random.shuffle(idx)
    X_shuf = X[idx]
    y_shuf = y[idx]
    
    # Split 4/5 train 1/5 test
    ntest = int(X_shuf.shape[0]/5)
    ntrain = int(X_shuf.shape[0]*4/5)
    X_train, X_test = X_shuf[:ntrain], X_shuf[:ntest]
    y_train, y_test = y_shuf[:ntrain], y_shuf[:ntest]
    
    print('X_train shape:',X_train.shape, 'X_test shape:',X_test.shape, 'y_train shape:',y_train.shape, 'y_test shape:',y_test.shape)

    print('Starting training loop...')
    for step in xrange(max_steps):
        Xb, Yb = next_batch(batch_size, X_train, y_train, is_augment)
        feed_dict = {xph: Xb, yph: Yb, learning_rate: lr}
        __, loss_train, acc_train = sess.run([train_op, loss, accuracy], feed_dict=feed_dict)

        if (step) % 5000 == 0 and step!=0 or (step + 1) == max_steps:                
            print('Step %d'%(step))
            print('Training Data Eval:')
            print ("accuracy training: " + "{:.5f}".format(acc_train))
            print ("loss training: " + "{:.5f}".format(loss_train))

            # Training Data Eval:
            ############
            nb_train_steps = int(math.ceil(float(y_train.shape[0])/(batch_size)))
            acc_train_all=0.
            for step_t in xrange(nb_train_steps):
                Xt=X_train[step_t*batch_size:(step_t+1)*batch_size]
                Yt=y_train[step_t*batch_size:(step_t+1)*batch_size]
                feed_dict = {xph: Xt, yph: Yt}
                accuracy_,pred_ = sess.run([accuracy,pred], feed_dict=feed_dict)
                acc_train_all += accuracy_*Xt.shape[0]
            acc_train_all=acc_train_all/X_train.shape[0]
            print ("accuracy training: " + "{:.5f}".format(acc_train_all))

            print('Test Data Eval:')
            ############
            nb_test_steps = int(math.ceil(float(y_test.shape[0])/(batch_size)))
            acc_test=0.
            for step_t in xrange(nb_test_steps):
                Xt=X_test[step_t*batch_size:(step_t+1)*batch_size]
                Yt=y_test[step_t*batch_size:(step_t+1)*batch_size]
                feed_dict = {xph: Xt, yph: Yt}
                accuracy_,pred_ = sess.run([accuracy,pred], feed_dict=feed_dict)
                acc_test += accuracy_*Xt.shape[0]
            acc_test=acc_test/X_test.shape[0]
            print ("accuracy test: " + "{:.5f}".format(acc_test))    
    list_test_acc.append(acc_test)
    print('list_test_acc: ',list_test_acc, np.mean(list_test_acc))
    sess.close()
